Import libraries

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
import numpy as np
import tensorflow as tf
import pickle
import re
from google.colab import drive
from keras.utils import np_utils

### Connect Google Drive Account
Please add files to the paths.

In [2]:
#add corpus file to your google drive path is 'Colab Notebooks/data/corpus_out3.txt'
#add test_sentence file to your google drive path is 'Colab Notebooks/data/test_sentences.txt'

drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks/data/corpus_out3.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Clean data and remove noisy

In [3]:
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)


file = open(path, 'r', encoding='utf-8')
data1 = file.read()
data1 = remove_URL(data1)
data1 = remove_html(data1)
data1 = data1.lower()
data1 = data1.replace('\n', '').replace('\r', '')
data = data1.split(".")
data = np.array_split(data, 16)[0]

print(len(data))
print(data[:10])

3425
['bilgisayar işletim sistemlerinin en temel parçası olan çekirdek yazılımlarından bir tanesidir'
 ' gnu genel kamu lisansı ile sunulan ve linux vakfı çatısı altında geliştirilen bir özgür yazılım projesidir'
 ' linux ismi ilk geliştiricisi olan linus torvalds tarafından 1991 yılında verilmiştir'
 ' günümüzde süper bilgisayarlarda, akıllı cihazların ve internet altyapısında kullanılan cihazların işletim sistemlerinde yaygın olarak kullanılmaktadır'
 ' bunlardan en popüler olanı google tarafından geliştirilen android işletim sistemidir'
 'ayrıca linux ismi, bu çekirdek kullanılarak oluşturulan işletim sistemlerini genel anlamda tanımlamak için yaygın bir kısaltma olarak da kullanılmaktadır'
 ' örneğin linux çekirdeği ve gnu araçları bir araya getirilerek tam bir işletim sistemi olarak sunulduğunda gnu/linux dağıtımı olarak adlandırılır, ancak konuşma dilinde kısaca linux olarak ifade edilmektedir'
 'linux kelimesinin bu iki farklı kullanımının yol açabileceği karışıklıktan kaçınmak 

Split data into word tokens

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
vocab = tokenizer.word_index
seqs = tokenizer.texts_to_sequences(data)

Prepare train data

In [5]:
def prepare_sentence(seq, maxlen):
    
    x = []
    y = []
    for i, w in enumerate(seq):
        x_padded = pad_sequences([seq[:i]],
                                 maxlen=maxlen - 1,
                                 padding='pre')[0]  
        x.append(x_padded)
        y.append(w)
    return x, y


maxlen = max([len(seq) for seq in seqs])
x = []
y = []
for seq in seqs:
    x_windows, y_windows = prepare_sentence(seq, maxlen)
    x += x_windows
    y += y_windows
x = np.array(x)
y = np.array(y) - 1  
y = np.eye(len(vocab))[y]  
print(x.shape)

(50581, 188)


Tain model

In [6]:
model = Sequential()
model.add(Embedding(input_dim=len(vocab) + 1,                    
                    output_dim=5,  
                    input_length=maxlen - 1))  
model.add(LSTM(64))
model.add(Dense(len(vocab), activation='softmax'))
model.compile('rmsprop', 'categorical_crossentropy')

model.fit(x, y, epochs=1)


#path='/content/drive/MyDrive/Colab Notebooks/model2/'
#pickle.dump(model, open(path+'model30.pkl','wb'))

317/317 [==============================] - 121s 371ms/step - loss: 8.7267 - val_loss: 9.0575


INFO:tensorflow:Assets written to: ram://516d0140-df5a-47df-a0a2-3499cb856c4d/assets


INFO:tensorflow:Assets written to: ram://516d0140-df5a-47df-a0a2-3499cb856c4d/assets


Try to Find best sentences

In [16]:
import sys
name1 = '/content/drive/My Drive/Colab Notebooks/data/test_sentences.txt'
in_testFile = open(name1, 'r', encoding='utf-8')

english = ["i", "o", "u", "s", "c", "g"]
turkish = {"i": "ı", "o": "ö", "u": "ü",
                  "s": "ş", "c": "ç", "g": "ğ"}

maxlen = max([len(seq) for seq in seqs])

def prepare_sentence(seq, maxlen):
    
    x = []
    y = []
    for i, w in enumerate(seq):
        x_padded = pad_sequences([seq[:i]],
                                 maxlen=maxlen - 1,
                                 padding='pre')[0]  
        x.append(x_padded)
        y.append(w)
    return x, y

def findPossibleSentences(sentence):

    occurenceDict = dict()
    resultSet = set()
    resultSet.add(sentence)
    size = 0
    for ele in english:
        occurenceDict[ele] = [m.start() for m in re.finditer(ele, sentence)]
        size = size + len(occurenceDict[ele])

    for key1, value1 in occurenceDict.items():
        if len(value1) > 0:
            for index in value1:
                tempList = []
                for sent in resultSet:
                    liste = list(sent)
                    liste[index] = turkish[key1]
                    tempList.append(''.join(liste))
                for item in tempList:
                    resultSet.add(item)

    return resultSet

def writePerpToFile(sentence):
    print("Best Sentence: " + sentence)
    print("------------------------------------------------")
  
def findBestSentence():
    allSentences = in_testFile.readlines()

    for sent in allSentences:
        possibles = findPossibleSentences(sent)
        print("Test Sentence: " + sent + "Number of Possible Sentences: " + str(len(possibles)))
        
        res = sent
        min = get_results(res)
        for item in possibles:
            temp = get_results(item)
            if  temp > min:
                min = temp
                res = item
        writePerpToFile(res)
            
def get_results(sentence):
    tok = tokenizer.texts_to_sequences([sentence])[0]
    x_test, y_test = prepare_sentence(tok, maxlen)
    if len(x_test)== 0 :
      return -sys.maxsize - 1
    else:
      x_test = np.array(x_test)
      y_test = np.array(y_test) - 1  
      
      p_pred = model.predict(x_test)
      
      
      vocab_inv = {v: k for k, v in vocab.items()}
      log_p_sentence = 0
      for i, prob in enumerate(p_pred):
          word = vocab_inv[y_test[i]+1]  
          prob_word = prob[y_test[i]]
          log_p_sentence += np.log(prob_word)
      return np.exp(log_p_sentence)
    


findBestSentence()

Test Sentence: bilgisayar isletim sistemlerinin 
Number of Possible Sentences: 4096
Best Sentence: bilgisayar işletım şistemlerınin 

------------------------------------------------
Test Sentence: ticari linux dagitimlarinin 
Number of Possible Sentences: 1024
Best Sentence: tiçarı linux dağitimlarinin 

------------------------------------------------
Test Sentence: orta duzeyde zengin bir ailede 
Number of Possible Sentences: 64
Best Sentence: örta duzeyde zenğin bir aılede 

------------------------------------------------
Test Sentence: yeni adiyla turkiye 
Number of Possible Sentences: 16
Best Sentence: yeni adiyla turkiye 

------------------------------------------------
Test Sentence: ilde bircok etkinlik 
Number of Possible Sentences: 64
Best Sentence: ilde birçok etkınlik 

------------------------------------------------
Test Sentence: cok sayida koyun yetistirilir
Number of Possible Sentences: 2048
Best Sentence: çok şayıda köyün yetıştırilir

-----------------------------